In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TFTrainer, TFTrainingArguments
from datasets import Dataset
import json
import tensorflow as tf

/Users/michelle/Desktop/LCT/Year1-CharlesUniversity/Summer Term/Statistical Machine Translation/AutoMin2023/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tokenizer = AutoTokenizer.from_pretrained("vmarklynn/bart-large-cnn-icsi-ami-v3")
# model = AutoModelForSeq2SeqLM.from_pretrained("vmarklynn/bart-large-cnn-icsi-ami-v3")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("MingZhong/DialogLED-base-16384")
model = AutoModelForSeq2SeqLM.from_pretrained("MingZhong/DialogLED-base-16384")

In [3]:
train_preprocess_file_path = '../preprocessed_data/europarl_transcripts_preprocessed_train.json'
dev_preprocess_file_path = '../preprocessed_data/europarl_transcripts_preprocessed_dev.json'

In [4]:
with open(train_preprocess_file_path, 'r') as f:
    europarl_train = json.loads(f.read())

with open(dev_preprocess_file_path, 'r') as f:
    europarl_dev = json.loads(f.read())

In [5]:
train_transcripts = []
train_minutes = []
for example in europarl_train:
    train_transcripts.append(example['transcript'])
    train_minutes.append(example['minutes'])
    

In [6]:
dev_transcripts = []
dev_minutes = []
for example in europarl_dev:
    dev_transcripts.append(example['transcript'])
    dev_minutes.append(example['minutes'])

In [7]:
train_transcript_encodings = tokenizer(train_transcripts, truncation=True, padding=True)
dev_transcript_encodings = tokenizer(dev_transcripts, truncation=True, padding=True)

In [8]:
train_minute_encodings = tokenizer(train_minutes, truncation=True, padding=True)
dev_minute_encodings = tokenizer(dev_minutes, truncation=True, padding=True)

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_transcript_encodings),
    dict(train_minute_encodings)
))
dev_dataset = tf.data.Dataset.from_tensor_slices((
    dict(dev_transcript_encodings),
    dict(dev_minute_encodings)
))


In [10]:
train_dataset

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(1024,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(1024,), dtype=tf.int32, name=None)}, {'input_ids': TensorSpec(shape=(1024,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(1024,), dtype=tf.int32, name=None)})>

In [14]:
training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)


In [13]:
trainer.train()

2023-04-19 14:36:39.049169: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_4"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
input: "Placeholder/_3"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 2065
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 1024
        }
      }
      shape {
        dim {
          size: 1024
        }
      }
      shape {
        dim {
          size: 1024
        }
      }
      

ValueError: `Checkpoint` was expecting model to be a trackable object (an object derived from `Trackable`), got BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
    